# Tidy Data
En esta seccion nos efocaremos al processo de limplieza y transformaccion para tener nuestro datos tidy

### Importando librerias

In [2]:
#Datos
import pandas as pd
import numpy as np
#Limpieza
import missingno as msno
#Archivos
import os, zipfile, requests, io
#plot
import seaborn as sns


## Descarga de datos

Los datos descargado para el proyecto son:
- 1.- worldbank - Niveles de educacion, Taza de alfabetizacion
- 2.- transparenciapresupuestaria - Presupuesto de la nacion

In [72]:
#Variables para descarga de datos

#Url de Educacion Banco de Mexico Informacion
url_carpeta_educacion = './raw_data/educacion/'
url_educacion_descarga = 'https://api.worldbank.org/v2/es/topic/4?downloadformat=csv'
url_educacion_local = url_carpeta_educacion+'API_4_DS2_es_csv_v2_3377123.csv'
file = os.path.exists(path=url_educacion_local)

if file == False:
    r = requests.get(url_educacion_descarga)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(url_carpeta_educacion)

    df_educacion = pd.read_csv(url_educacion_local, sep=',',skiprows=4,index_col=False)

else:
    df_educacion = pd.read_csv(url_educacion_local, sep=',',skiprows=4,index_col=False)



In [102]:
#Url presupuesto de la Nacion Historial
url_carpeta_presupuesto = './raw_data/presupuesto/'
url_descarga_presupuesto = 'http://s3.amazonaws.com/datastore.openspending.org/667df60aa07c34260eae9b55b2778712/presupuesto_mexico_2008_2019/final/data/presupuesto_mexico_2008_2019.csv'
url_presupuesto_local = url_carpeta_presupuesto+'presupuesto_mexico_2008_2019.csv'

file = os.path.exists(path=url_presupuesto_local)

if file == False:
    try:
        r = requests.get(url_descarga_presupuesto)
        #codificacion de archivo
        decoded_content = r.content.decode('utf-8')
        csv_file = open(url_presupuesto_local, 'wb')
        csv_file.write(bytes(decoded_content,encoding='utf8'))
        csv_file.close()
    except Exception as e:
        print("Algo salio mal: ",e)
        csv_file.close()

    df_presupuesto = pd.read_csv(url_presupuesto_local, sep=',',index_col=False)

else:
    df_presupuesto = pd.read_csv(url_presupuesto_local, sep=',',index_col=False)



C:\Users\ibsan\anaconda3\envs\envname\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (8,15,19,28,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Educacion - Dataset
Se revisan la estrutura de datos, se selecciona, limpia y transforma - Educacion

In [105]:
# Seleccion de datos mexicanos
#Remover columnas no necesarias
df_educacion_mexico = df_educacion[df_educacion['Country Code'] =='MEX']
#renombrar columna
df_educacion_mexico.rename(columns={'Indicator Name':'Año'}, inplace= True)
#Cambion de index
df_educacion_mexico = df_educacion_mexico.drop(['Country Name','Country Code','Indicator Code'],axis=1)

df_educacion_mexico.set_index('Año',inplace=True)
#Se trapuso la matriz
df_educacion_mexico_transpo = df_educacion_mexico.T
df_educacion_mexico_transpo.reset_index(inplace=True)

df_educacion_mexico_transpo.rename(columns={'index':'Fecha'}, inplace= True)
df_educacion_mexico_transpo.rename_axis(None, axis = 1,inplace=True)
#Remove last row
df_educacion_mexico_transpo.drop(df_educacion_mexico_transpo.index[-1], inplace=True)
#String a Fecha
df_educacion_mexico_transpo['Fecha'] = df_educacion_mexico_transpo['Fecha'].astype('object') #pd.to_datetime(df_educacion_mexico_transpo['Fecha'],format='%Y')
#Filtrado de registros con fechas de 2008 a 2019
df_educacion_mexico_2008_2019= df_educacion_mexico_transpo[(df_educacion_mexico_transpo['Fecha'] >= '2008') & (df_educacion_mexico_transpo['Fecha'] <= '2019')]
df_educacion_mexico_2008_2019.reset_index(drop=True, inplace=True)


### Seleccion de Columnas
Seleccionamos algunas columnas que consideramos mas relevantes para nuesto proyecto 

In [106]:
listColumn = ['Fecha',
'Tasa de alfabetización, total de adultos (% de personas de 15 años o más)',
'Tasa de alfabetización, varones adultos (% de varones de 15 años o más)',
'Tasa de alfabetización, mujeres adultas (% de mujeres de 15 años o más)',
'Nivel de instrucción, al menos nivel de maestría o equivalente, población de más de 25 años, total (%) (acumulativo)',
'Nivel de instrucción, al menos nivel de maestría o equivalente, población de más de 25 años, varones (%) (acumulativo)',
'Nivel de instrucción, al menos nivel de maestría o equivalente, población de más de 25 años, mujeres (%) (acumulativo)',
'Nivel de instrucción, al menos nivel de doctorado o equivalente, población de más de 25 años, total (%) (acumulativo)',
'Nivel de instrucción, al menos nivel de doctorado o equivalente, población de más de 25 años, varones (%) (acumulativo)',
'Nivel de instrucción, al menos nivel de doctorado o equivalente, población de más de 25 años, mujeres (%) (acumulativo)',
'Nivel de instrucción, al menos nivel de licenciatura o equivalente, población de más de 25 años, total (%) (acumulativo)',
'Nivel de instrucción, al menos nivel de licenciatura o equivalente, población de más de 25 años, varones (%) (acumulativo)',
]
df_educacion_mexico_2008_2019_Main = df_educacion_mexico_2008_2019[listColumn]

df_educacion_mexico_2008_2019_Main

,Fecha,"Tasa de alfabetización, total de adultos (% de personas de 15 años o más)","Tasa de alfabetización, varones adultos (% de varones de 15 años o más)","Tasa de alfabetización, mujeres adultas (% de mujeres de 15 años o más)","Nivel de instrucción, al menos nivel de maestría o equivalente, población de más de 25 años, total (%) (acumulativo)","Nivel de instrucción, al menos nivel de maestría o equivalente, población de más de 25 años, varones (%) (acumulativo)","Nivel de instrucción, al menos nivel de maestría o equivalente, población de más de 25 años, mujeres (%) (acumulativo)","Nivel de instrucción, al menos nivel de doctorado o equivalente, población de más de 25 años, total (%) (acumulativo)","Nivel de instrucción, al menos nivel de doctorado o equivalente, población de más de 25 años, varones (%) (acumulativo)","Nivel de instrucción, al menos nivel de doctorado o equivalente, población de más de 25 años, mujeres (%) (acumulativo)","Nivel de instrucción, al menos nivel de licenciatura o equivalente, población de más de 25 años, total (%) (acumulativo)","Nivel de instrucción, al menos nivel de licenciatura o equivalente, población de más de 25 años, varones (%) (acumulativo)"
0,2008,92.925598,94.592163,91.453613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009,93.441879,94.916573,92.116249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,93.068939,94.383942,91.853416,1.31121,1.55810,1.08774,0.14268,0.19103,0.09891,12.76122,14.069390
3,2011,93.519981,94.782768,92.338280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012,94.228401,95.391632,93.179237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2013,93.962852,95.112106,92.903938,1.28283,1.56697,1.03394,0.08504,0.11892,0.05535,14.32261,15.869660
6,2014,94.555878,95.620468,93.591339,1.24434,1.47556,1.04288,0.08402,0.12969,0.04421,13.95713,15.228270
7,2015,94.472282,95.549332,93.485497,2.08774,2.44526,1.76869,0.22477,0.28954,0.16699,14.77965,16.089300
8,2016,94.859619,95.844818,93.968521,1.39542,1.55096,1.25950,0.07626,0.10133,0.05435,14.71039,16.086281
9,2017,94.972778,95.872368,94.165756,1.44282,1.60556,1.30201,0.07681,0.10201,0.05501,15.37426,16.966221


Verificamos la forma de nuestro dataset para trabajar

In [75]:
df_educacion_mexico_2008_2019_Main.shape

(12, 12)

Vemos los valores nulos o si definir de cada columna 

In [76]:
df_educacion_mexico_2008_2019_Main.isna().sum()

Fecha                                                                                                                         0
Tasa de alfabetización, total de adultos (% de personas de 15 años o más)                                                     1
Tasa de alfabetización, varones adultos (% de varones de 15 años o más)                                                       1
Tasa de alfabetización, mujeres adultas (% de mujeres de 15 años o más)                                                       1
Nivel de instrucción, al menos nivel de maestría o equivalente, población de más de 25 años, total (%) (acumulativo)          5
Nivel de instrucción, al menos nivel de maestría o equivalente, población de más de 25 años, varones (%) (acumulativo)        5
Nivel de instrucción, al menos nivel de maestría o equivalente, población de más de 25 años, mujeres (%) (acumulativo)        5
Nivel de instrucción, al menos nivel de doctorado o equivalente, población de más de 25 años, total (%) 

Decidimos eliminar el registro del 2019 debido que es un reglon sin datos

In [77]:
df_educacion_mexico_2008_2019_Main.drop(df_educacion_mexico_2008_2019_Main.index[-1], inplace=True)

C:\Users\ibsan\anaconda3\envs\envname\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Validamos los tipos de datos 

In [79]:
df_educacion_mexico_2008_2019_Main.dtypes

,Fecha,"Tasa de alfabetización, total de adultos (% de personas de 15 años o más)","Tasa de alfabetización, varones adultos (% de varones de 15 años o más)","Tasa de alfabetización, mujeres adultas (% de mujeres de 15 años o más)","Nivel de instrucción, al menos nivel de maestría o equivalente, población de más de 25 años, total (%) (acumulativo)","Nivel de instrucción, al menos nivel de maestría o equivalente, población de más de 25 años, varones (%) (acumulativo)","Nivel de instrucción, al menos nivel de maestría o equivalente, población de más de 25 años, mujeres (%) (acumulativo)","Nivel de instrucción, al menos nivel de doctorado o equivalente, población de más de 25 años, total (%) (acumulativo)","Nivel de instrucción, al menos nivel de doctorado o equivalente, población de más de 25 años, varones (%) (acumulativo)","Nivel de instrucción, al menos nivel de doctorado o equivalente, población de más de 25 años, mujeres (%) (acumulativo)","Nivel de instrucción, al menos nivel de licenciatura o equivalente, población de más de 25 años, total (%) (acumulativo)","Nivel de instrucción, al menos nivel de licenciatura o equivalente, población de más de 25 años, varones (%) (acumulativo)"
0,2008,92.925598,94.592163,91.453613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009,93.441879,94.916573,92.116249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,93.068939,94.383942,91.853416,1.31121,1.55810,1.08774,0.14268,0.19103,0.09891,12.76122,14.069390
3,2011,93.519981,94.782768,92.338280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012,94.228401,95.391632,93.179237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2013,93.962852,95.112106,92.903938,1.28283,1.56697,1.03394,0.08504,0.11892,0.05535,14.32261,15.869660
6,2014,94.555878,95.620468,93.591339,1.24434,1.47556,1.04288,0.08402,0.12969,0.04421,13.95713,15.228270
7,2015,94.472282,95.549332,93.485497,2.08774,2.44526,1.76869,0.22477,0.28954,0.16699,14.77965,16.089300
8,2016,94.859619,95.844818,93.968521,1.39542,1.55096,1.25950,0.07626,0.10133,0.05435,14.71039,16.086281
9,2017,94.972778,95.872368,94.165756,1.44282,1.60556,1.30201,0.07681,0.10201,0.05501,15.37426,16.966221


## Presupuesto - Dataset
Se revisan la estrutura de datos, se selecciona, limpia y transforma - Presupuesto de la nacion

In [107]:
#Verificamos los primeros registros
df_presupuesto.head()

,CICLO,DESC_AI,DESC_CAPITULO,DESC_CONCEPTO,DESC_FF,DESC_FUNCION,DESC_GPO_FUNCIONAL,DESC_MODALIDAD,DESC_PARTIDA_ESPECIFICA,DESC_PARTIDA_GENERICA,...,ID_SUBFUNCION,ID_TIPOGASTO,ID_UR,MONTO_ADEFAS,MONTO_APROBADO,MONTO_DEVENGADO,MONTO_EJERCICIO,MONTO_EJERCIDO,MONTO_MODIFICADO,MONTO_PAGADO
0,2008,Fiscalización de la Cuenta de la Hacienda Públ...,Servicios personales,Remuneraciones al personal de carácter permanente,Recursos fiscales,Legislación,Gobierno,Otras actividades relevantes,NaN,Sueldos base,...,1,1,101,NaN,111993252.0,NaN,NaN,128276238.0,NaN,NaN
1,2008,Fiscalización de la Cuenta de la Hacienda Públ...,Servicios personales,Remuneraciones al personal de carácter transit...,Recursos fiscales,Legislación,Gobierno,Otras actividades relevantes,NaN,Honorarios,...,1,1,101,NaN,21791544.0,NaN,NaN,70124960.0,NaN,NaN
2,2008,Fiscalización de la Cuenta de la Hacienda Públ...,Servicios personales,Remuneraciones adicionales y especiales,Recursos fiscales,Legislación,Gobierno,Otras actividades relevantes,NaN,Prima quinquenal por años de servicios efectiv...,...,1,1,101,NaN,915708.0,NaN,NaN,894256.0,NaN,NaN
3,2008,Fiscalización de la Cuenta de la Hacienda Públ...,Servicios personales,Remuneraciones adicionales y especiales,Recursos fiscales,Legislación,Gobierno,Otras actividades relevantes,NaN,Primas de vacaciones y dominical,...,1,1,101,NaN,3272498.0,NaN,NaN,3643226.0,NaN,NaN
4,2008,Fiscalización de la Cuenta de la Hacienda Públ...,Servicios personales,Remuneraciones adicionales y especiales,Recursos fiscales,Legislación,Gobierno,Otras actividades relevantes,NaN,Gratificación de fin de año,...,1,1,101,NaN,52316091.0,NaN,NaN,59948772.0,NaN,NaN


Damos un vistaso a los typos de datos que tenemos

In [108]:
df_presupuesto.dtypes

CICLO                        int64
DESC_AI                     object
DESC_CAPITULO               object
DESC_CONCEPTO               object
DESC_FF                     object
DESC_FUNCION                object
DESC_GPO_FUNCIONAL          object
DESC_MODALIDAD              object
DESC_PARTIDA_ESPECIFICA     object
DESC_PARTIDA_GENERICA       object
DESC_PP                     object
DESC_RAMO                   object
DESC_SUBFUNCION             object
DESC_TIPOGASTO              object
DESC_UR                     object
ENTIDAD_FEDERATIVA          object
GPO_FUNCIONAL                int64
ID_AI                        int64
ID_CAPITULO                  int64
ID_CLAVE_CARTERA            object
ID_CONCEPTO                  int64
ID_ENTIDAD_FEDERATIVA      float64
ID_FF                        int64
ID_FUNCION                   int64
ID_MODALIDAD                object
ID_PARTIDA_ESPECIFICA      float64
ID_PARTIDA_GENERICA          int64
ID_PP                        int64
ID_RAMO             

Forma de dataset

In [113]:
df_presupuesto.shape

(2767466, 39)

In [ ]:
df_presupuesto.dtypes

Para cuestiones del proyecto no enfocamos en el ramo 11 (Educación Pública) segun catalago presupuestario dentro de la carpeta raw_data/presupuesto, catalogos_presupuestarios.csv hoja - Ramo, seleccionamos Funcion 

In [169]:
df_presupuesto['ID_RAMO'] = df_presupuesto['ID_RAMO'].astype('category')
df_presupuesto_educacion = df_presupuesto.loc[df_presupuesto['DESC_RAMO']=='Educación Pública'] #& (df_presupuesto['ID_FUNCION']==5)

In [170]:
df_presupuesto_educacion_short = df_presupuesto_educacion[['CICLO','ID_RAMO','DESC_RAMO','ID_TIPOGASTO','DESC_TIPOGASTO','MONTO_APROBADO','MONTO_EJERCICIO']]
df_presupuesto_educacion_short['CICLO'] = df_presupuesto_educacion_short['CICLO'].astype('category')#pd.to_datetime(df_presupuesto_educacion_short['CICLO'],format='%Y')
df_presupuesto_educacion_short['ID_TIPOGASTO'] =df_presupuesto_educacion_short['ID_TIPOGASTO'].astype('object')

C:\Users\ibsan\AppData\Local\Temp/ipykernel_40556/2904175776.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_presupuesto_educacion_short['CICLO'] = df_presupuesto_educacion_short['CICLO'].astype('category')#pd.to_datetime(df_presupuesto_educacion_short['CICLO'],format='%Y')
C:\Users\ibsan\AppData\Local\Temp/ipykernel_40556/2904175776.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_presupuesto_educacion_short['ID_TIPOGASTO'] =df_presupuesto_educacion_short['ID_TIPOGASTO'].astype('object')


Vemos la forma de nuestro dataset

In [171]:
df_presupuesto_educacion_short.shape

(149123, 7)

Obtener valores nulos, lo que nos muestra un set de datos completos

In [172]:
df_presupuesto_educacion_short.isna().sum()

CICLO                  0
ID_RAMO                0
DESC_RAMO              0
ID_TIPOGASTO           0
DESC_TIPOGASTO         0
MONTO_APROBADO         0
MONTO_EJERCICIO    76921
dtype: int64

Agrupamos nuestro dato set por tipo de gasto

In [173]:
df_presupuesto_educacion_ciclo = df_presupuesto_educacion_short.groupby(['CICLO','DESC_RAMO','DESC_TIPOGASTO'])['MONTO_APROBADO','MONTO_EJERCICIO'].sum()
df_presupuesto_educacion_ciclo.reset_index( inplace=True)


C:\Users\ibsan\AppData\Local\Temp/ipykernel_40556/4243046433.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_presupuesto_educacion_ciclo = df_presupuesto_educacion_short.groupby(['CICLO','DESC_RAMO','DESC_TIPOGASTO'])['MONTO_APROBADO','MONTO_EJERCICIO'].sum()


,CICLO,DESC_RAMO,DESC_TIPOGASTO,MONTO_APROBADO,MONTO_EJERCICIO
0,2008,Educación Pública,Gasto corriente,1.619434e+11,0.0
1,2008,Educación Pública,Gasto corriente por concepto de gastos indirec...,0.000000e+00,0.0
2,2008,Educación Pública,Gasto corriente por concepto de recursos otorg...,0.000000e+00,0.0
3,2008,Educación Pública,Gasto de capital diferente de obra pública,8.656397e+09,0.0
4,2008,Educación Pública,Gasto de inversión por concepto de gastos indi...,0.000000e+00,0.0
...,...,...,...,...,...
67,2019,Educación Pública,Gasto corriente por concepto de gastos indirec...,1.797463e+08,0.0
68,2019,Educación Pública,Gasto corriente por concepto de recursos otorg...,0.000000e+00,0.0
69,2019,Educación Pública,Gasto de capital diferente de obra pública,2.680839e+08,0.0
70,2019,Educación Pública,Gasto de inversión por concepto de gastos indi...,0.000000e+00,0.0


Eliminamos las columna Desc_ramo ya que tenemos registro identicos

In [177]:
df_presupuesto_educacion_ciclo_tidy = df_presupuesto_educacion_ciclo[['CICLO','DESC_TIPOGASTO','MONTO_APROBADO','MONTO_EJERCICIO']]
df_presupuesto_educacion_ciclo_tidy.shape

(72, 4)

Por ultimo removemos los datos de monto_aprovado en zeros

In [180]:
df_presupuesto_educacion_ciclo_tidy = df_presupuesto_educacion_ciclo_tidy[df_presupuesto_educacion_ciclo_tidy['MONTO_APROBADO']>0]
df_presupuesto_educacion_ciclo_tidy.shape
df_presupuesto_educacion_ciclo_tidy.reset_index(drop=True, inplace=True)
df_presupuesto_educacion_ciclo_tidy

,CICLO,DESC_TIPOGASTO,MONTO_APROBADO,MONTO_EJERCICIO
0,2008,Gasto corriente,1.619434e+11,0.000000e+00
1,2008,Gasto de capital diferente de obra pública,8.656397e+09,0.000000e+00
2,2008,Gasto de obra pública,2.898005e+09,0.000000e+00
3,2009,Gasto corriente,1.904939e+11,0.000000e+00
4,2009,Gasto de capital diferente de obra pública,3.834694e+09,0.000000e+00
5,2009,Gasto de obra pública,6.601937e+09,0.000000e+00
6,2010,Gasto corriente,2.007101e+11,0.000000e+00
7,2010,Gasto corriente por concepto de gastos indirec...,1.913210e+09,0.000000e+00
8,2010,Gasto de capital diferente de obra pública,3.527667e+09,0.000000e+00
9,2010,Gasto de inversión por concepto de gastos indi...,1.875891e+08,0.000000e+00


### Deteccion de anomalias 
Para la deteccion de anomalias queremos comprobar si el gasto publico tiene un tendencia y si esta tiene algun comportamiento diferente.
- EllipticEnvelope

In [ ]:
from sklearn.covariance import EllipticEnvelope